# Generating result

## Set model

In [1]:
!pip install -q -U google-generativeai

In [2]:
import pathlib
import google.generativeai as genai

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
model = genai.GenerativeModel('gemini-1.0-pro-001')

## generate text response/chat response and write it to csv

# function

In [6]:
import os
import pandas as pd
from tqdm import tqdm
import time
from vertexai.preview.generative_models import (
    HarmCategory,
    HarmBlockThreshold )
from google.cloud.aiplatform_v1beta1.types.content import SafetySetting

def getGemini10Pro001Response(prompt):
    safety_settings : list[str] = [{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}]
    response = model.generate_content(prompt,safety_settings = safety_settings).text
    return response

def getLLMResponse(prompt):
    return getGemini10Pro001Response(prompt)

def splitQuestion(question):
    sentences = question.split('. ')
    last_sentence = sentences[-1]
    # Remove the last sentence
    sentences.pop(-1)
    # Join the remaining
    new_question = '. '.join(sentences)
    return last_sentence, new_question
def getGeminiChatResponseNoSub(model,context,question): #model type / large question in str/ subquestions in array
    header_instruction = ''' Consider this fictional board game scenario and the rules defined for the scenario: ''' ##### the one for starting the chat
    # header_last_instruction=''' Given this, wait for the subproblems I give you and answer them accordingly   '''
    final_instruction = ''' Given this, answer the final question: ''' ### what's the best answer etc...
    # last_sentence, new_question = splitQuestion(question)
    safety_settings : list[str] = [{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}]
    chat = model.start_chat(history=[])
    response = chat.send_message(header_instruction + context + final_instruction + question,
                                 safety_settings=safety_settings)
    # return response.text
    history =""
    for message in chat.history:
        history += message.role + ": " + message.parts[0].text + "\n"
    return history, response.text

def getGeminiChatResponse(model,context,question,subquestions): #model type / large question in str/ subquestions in array
    header_instruction = ''' Consider this fictional board game scenario and the rules defined for the scenario: ''' ##### the one for starting the chat
    header_last_instruction=''' Given this, wait for the subproblems I give you and answer them accordingly ok?  '''
    final_instruction = ''' Given the answers you have generated for the subproblems, answer the final question: ''' ### what's the best answer etc...
    # last_sentence, new_question = splitQuestion(question)
    safety_settings : list[str] = [{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}]
    chat = model.start_chat(history=[])
    response = chat.send_message(header_instruction + context +header_last_instruction,safety_settings=safety_settings )
    for subs in subquestions:
        response = chat.send_message(subs,safety_settings=safety_settings)
    response  = chat.send_message(final_instruction + question,safety_settings=safety_settings)
    # return response.text
    history =""
    for message in chat.history:
        history += message.role + ": " + message.parts[0].text + "\n"
    return history, response.text

In [59]:
import csv
import json
def getBaseAnswers(dataDf):
    output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_results.csv'
    #output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_sub_generation.csv'
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Example', 'Response'])
        count = 0
        for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
            prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
            #prompt = "Based on this question: "+row['example'] + "\n Could you generate some sub-questions?"+ "\n each sub-problem should be divided by '||' "
            response = getLLMResponse(prompt)
            writer.writerow([row['example'], response])
            count += 1
            print("The response for " + str(count) + "is: ",response)
def getSubs(dataDf):
    # output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_results.csv'
    output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_sub_generation.csv'
    with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # writer.writerow(['Example', 'Response'])
        count = 0
        for index, row in tqdm(dataDf.iloc[977:].iterrows(), total=dataDf.shape[0]-977):
            # prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
            prompt = "Based on this question: "+row['example'] + "\n Could you generate some sub-questions?"+ '''\n
            the response should follow the format: \n
            1: (subquestion1) \n
            2: (subquestion2) \n
            3: (subquestion1) \n
            and so on......'''
            response = getLLMResponse(prompt)
            writer.writerow([row['example'], response])
            count += 1
            # print("The response for " + str(count) + "is: ",response)

def getChatResponse(dataDf):
    # output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_final_results.csv'
    output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv'
    with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # writer.writerow(['context','question','answer', 'chat'])
        count = 0
        for index, row in tqdm(dataDf.iloc[924:].iterrows(), total=dataDf.shape[0]-924):
            context = row['context']
            question = row['question'] + "\n Give me a one word answer (proved, disproved, unknown)"
            subquestion = row["parsed_Response"]
            if not subquestion or subquestion == "nan":
                print("no subs")
                history,response = getGeminiChatResponseNoSub(model,context,question)
            else:
                subquestions = subquestion.split("||")
                print("have subs")
                history,response = getGeminiChatResponse(model,context,question,subquestions)
            writer.writerow([context, question,response,history])
            count += 1
            # print("The response for " + str(count) + "is: ",response)
def getCOTAnswers(dataDf):
    output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_CoT_results.csv'
    CoT = '''
        Given the question:
            A few players are playing a boardgame. The current state of the game is as follows. The carp rolls the dice for the squirrel. The sea bass sings a victory song for the elephant. And the rules of the game are as follows. Rule1: If the carp rolls the dice for the squirrel, then the squirrel removes from the board one of the pieces of the sheep. Rule2: If the squirrel removes one of the pieces of the sheep and the zander knocks down the fortress of the sheep, then the sheep removes one of the pieces of the whale. Rule3: Regarding the zander, if it took a bike from the store, then we can conclude that it does not knock down the fortress of the sheep. Rule4: The zander knocks down the fortress of the sheep whenever at least one animal sings a song of victory for the elephant. Rule3 is preferred over Rule4. Based on the game state and the rules and preferences, does the sheep remove from the board one of the pieces of the whale?
        The reasoning is:
            We know the sea bass sings a victory song for the elephant, and according to Rule4 "if at least one animal sings a victory song for the elephant, then the zander knocks down the fortress of the sheep", and for the conflicting and higher priority rule Rule3 we cannot prove the antecedent "the zander took a bike from the store", so we can conclude "the zander knocks down the fortress of the sheep". We know the carp rolls the dice for the squirrel, and according to Rule1 "if the carp rolls the dice for the squirrel, then the squirrel removes from the board one of the pieces of the sheep", so we can conclude "the squirrel removes from the board one of the pieces of the sheep". We know the squirrel removes from the board one of the pieces of the sheep and the zander knocks down the fortress of the sheep, and according to Rule2 "if the squirrel removes from the board one of the pieces of the sheep and the zander knocks down the fortress of the sheep, then the sheep removes from the board one of the pieces of the whale", so we can conclude "the sheep removes from the board one of the pieces of the whale". So the statement "the sheep removes from the board one of the pieces of the whale" is proved and the answer is "yes".
        The label is:
            proved
        Now giving the question: '''
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Example', 'Response'])
        count = 0
        for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
            prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
            response = getLLMResponse(prompt)
            writer.writerow([row['example'], response])
            count += 1
            print("The response for " + str(count) + "is: ",response)

# call

In [60]:
def main():
    # dataPath = "/content/drive/MyDrive/gemini/data/test.json"
    # df = pd.read_json(dataPath, dtype=str)
    # getBaseAnswers(df)
    #  dataPath = "/content/drive/MyDrive/gemini/data/output-with-subproblems.json"
    #  df = pd.read_json(dataPath, dtype=str)
    #  getChatResponse(df)
     dataPath = "/content/drive/MyDrive/gemini/data/Gemini_parsed_sub_generation.csv"
     df = pd.read_csv(dataPath)
     df = df.astype(str)
     getChatResponse(df)

if __name__ == "__main__":
    main()


  0%|          | 0/76 [00:00<?, ?it/s]

have subs


  1%|▏         | 1/76 [00:16<20:50, 16.67s/it]

have subs


  3%|▎         | 2/76 [00:29<17:37, 14.28s/it]

...

 97%|█████████▋| 74/76 [20:24<00:29, 14.71s/it]

have subs


 99%|█████████▊| 75/76 [20:40<00:15, 15.06s/it]

have subs


100%|██████████| 76/76 [20:58<00:00, 16.56s/it]


In [58]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv')

# The length of the DataFrame is the number of rows excluding the header
row_count = len(df)

print(f"Number of rows excluding the header: {row_count}")


Number of rows excluding the header: 924


In [ ]:
def CoT():
    dataPath = "/content/drive/MyDrive/gemini/data/test.json"
    with open(dataPath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    getCOTAnswers(df)
CoT()

In [ ]:
def Generate_subs():
    dataPath = "/content/drive/MyDrive/gemini/data/test.json"
    with open(dataPath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    getSubs(df)
Generate_subs()

100%|██████████| 23/23 [00:58<00:00,  2.54s/it]


#Process Answer

## Check Header

### subs

In [ ]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_final_results.csv'

with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)
    print(f"Header: {header}")


Header: ['context', 'question', 'answer', 'chat']


### base

In [ ]:
file_path2 = '/content/drive/MyDrive/gemini/data/Gemini_results.csv'

with open(file_path2, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)
    print(f"Header: {header}")

Header: ['Example', 'Response']


### cot

In [ ]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_CoT_results.csv'

with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)
    print(f"Header: {header}")

Header: ['Example', 'Response']


### gene_subs

In [61]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv'

with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)
    print(f"Header: {header}")

Header: ['context', 'question', 'answer', 'chat']


## Change all answer to lower

### subs

In [ ]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_final_results.csv'
column_name = 'answer'

modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    headers = reader.fieldnames
    for row in reader:
        row[column_name] = row[column_name].lower()  # Convert to lowercase
        modified_rows.append(row)

with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(modified_rows)

### base

In [ ]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_results.csv'
column_name = 'Response'

modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    headers = reader.fieldnames
    for row in reader:
        row[column_name] = row[column_name].lower()  # Convert to lowercase
        modified_rows.append(row)

with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(modified_rows)

### cot

In [ ]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_CoT_results.csv'
column_name = 'Response'

modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    headers = reader.fieldnames
    for row in reader:
        row[column_name] = row[column_name].lower()  # Convert to lowercase
        modified_rows.append(row)

with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(modified_rows)

### gene_subs

In [62]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv'
column_name = 'answer'

modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    headers = reader.fieldnames
    for row in reader:
        row[column_name] = row[column_name].lower()  # Convert to lowercase
        modified_rows.append(row)

with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(modified_rows)

##Check the answer and update the csv

### subs

In [ ]:
import re
file_path = '/content/drive/MyDrive/gemini/data/Gemini_final_results.csv'
column_name = 'answer'
def contains_all_three(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_proved and has_disproved and has_unknown
def contains_both_proved_disproved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_proved and has_disproved
def contains_proved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    return has_proved
def contains_disproved(text):
    # Regular expression pattern to match whole words only
    disproved_pattern = r'\bdisproved\b'
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_disproved
def contains_unknown(text):
    # Regular expression pattern to match whole words only
    unknown_pattern = r'\bunknown\b'
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_unknown
def contains_none(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is None
    has_disproved = re.search(disproved_pattern, text) is None
    has_unknown = re.search(unknown_pattern, text) is None
    return has_proved and has_disproved and has_unknown
modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    countb = 0
    countp = 0
    countd = 0
    countu = 0
    counta = 0
    countn = 0
    for index, row in enumerate(reader, start=1):
        column_value = row[column_name]
        if contains_all_three(column_value):
            counta +=1
        if contains_none(column_value):
            print("the index for none: ",index)
            countn +=1
            row[column_name] = "proved"
        if contains_both_proved_disproved(column_value):
            countb +=1
        if contains_proved(column_value):
            countp +=1
            row[column_name] = "proved"
        if contains_disproved(column_value):
            countd +=1
            row[column_name] = "disproved"
        if contains_unknown(column_value):
            countu +=1
            row[column_name] = "unknown"
        modified_rows.append(row)
    print("all three: ",counta)
    print("none: ",countn)
    print("both p and d: ",countb)
    print("proved: ",countp)
    print("disproved: ",countd)
    print("unknown: ",countu)
    print("total: ",len(modified_rows))
with open("/content/drive/MyDrive/gemini/data/updated_final_results.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(modified_rows)

the index for none:  153
all three:  0
none:  1
both p and d:  0
proved:  286
disproved:  411
unknown:  302
total:  1000


### base

In [ ]:
import re
file_path = '/content/drive/MyDrive/gemini/data/Gemini_results.csv'
column_name = 'Response'
def contains_all_three(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_proved and has_disproved and has_unknown
def contains_both_proved_disproved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_proved and has_disproved
def contains_proved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    return has_proved
def contains_disproved(text):
    # Regular expression pattern to match whole words only
    disproved_pattern = r'\bdisproved\b'
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_disproved
def contains_unknown(text):
    # Regular expression pattern to match whole words only
    unknown_pattern = r'\bunknown\b'
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_unknown
def contains_none(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is None
    has_disproved = re.search(disproved_pattern, text) is None
    has_unknown = re.search(unknown_pattern, text) is None
    return has_proved and has_disproved and has_unknown
modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    countb = 0
    countp = 0
    countd = 0
    countu = 0
    counta = 0
    countn = 0
    for index, row in enumerate(reader, start=1):
        column_value = row[column_name]
        if contains_all_three(column_value):
            counta +=1
        if contains_none(column_value):
            countn +=1
        if contains_both_proved_disproved(column_value):
            countb +=1
        if contains_proved(column_value):
            countp +=1
            row[column_name] = "proved"
        if contains_disproved(column_value):
            countd +=1
            row[column_name] = "disproved"
        if contains_unknown(column_value):
            countu +=1
            row[column_name] = "unknown"
        modified_rows.append(row)
    print("all three: ",counta)
    print("none: ",countn)
    print("both p and d: ",countb)
    print("proved: ",countp)
    print("disproved: ",countd)
    print("unknown: ",countu)
    print("total: ",len(modified_rows))
with open("/content/drive/MyDrive/gemini/data/updated_gemini_results.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(modified_rows)

all three:  0
none:  0
both p and d:  0
proved:  357
disproved:  443
unknown:  200
total:  1000


### cot

In [ ]:
import re
file_path = '/content/drive/MyDrive/gemini/data/Gemini_CoT_results.csv'
column_name = 'Response'
def contains_all_three(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_proved and has_disproved and has_unknown
def contains_both_proved_disproved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_proved and has_disproved
def contains_proved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    return has_proved
def contains_disproved(text):
    # Regular expression pattern to match whole words only
    disproved_pattern = r'\bdisproved\b'
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_disproved
def contains_unknown(text):
    # Regular expression pattern to match whole words only
    unknown_pattern = r'\bunknown\b'
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_unknown
def contains_none(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is None
    has_disproved = re.search(disproved_pattern, text) is None
    has_unknown = re.search(unknown_pattern, text) is None
    return has_proved and has_disproved and has_unknown
modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    countb = 0
    countp = 0
    countd = 0
    countu = 0
    counta = 0
    countn = 0
    for index, row in enumerate(reader, start=1):
        column_value = row[column_name]
        if contains_all_three(column_value):
            counta +=1
        if contains_none(column_value):
            countn +=1
        if contains_both_proved_disproved(column_value):
            countb +=1
        if contains_proved(column_value):
            countp +=1
            row[column_name] = "proved"
        if contains_disproved(column_value):
            countd +=1
            row[column_name] = "disproved"
        if contains_unknown(column_value):
            countu +=1
            row[column_name] = "unknown"
        modified_rows.append(row)
    print("all three: ",counta)
    print("none: ",countn)
    print("both p and d: ",countb)
    print("proved: ",countp)
    print("disproved: ",countd)
    print("unknown: ",countu)
    print("total: ",len(modified_rows))
with open("/content/drive/MyDrive/gemini/data/Gemini_parsed_CoT_results.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(modified_rows)

all three:  0
none:  0
both p and d:  0
proved:  331
disproved:  448
unknown:  221
total:  1000


### gene_subs

In [75]:
import re
file_path = '/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv'
column_name = 'answer'
def contains_all_three(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_proved and has_disproved and has_unknown
def contains_both_proved_disproved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_proved and has_disproved
def contains_proved(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    has_proved = re.search(proved_pattern, text) is not None
    return has_proved
def contains_disproved(text):
    # Regular expression pattern to match whole words only
    disproved_pattern = r'\bdisproved\b'
    has_disproved = re.search(disproved_pattern, text) is not None
    return has_disproved
def contains_unknown(text):
    # Regular expression pattern to match whole words only
    unknown_pattern = r'\bunknown\b'
    has_unknown = re.search(unknown_pattern, text) is not None
    return has_unknown
def contains_none(text):
    # Regular expression pattern to match whole words only
    proved_pattern = r'\bproved\b'
    disproved_pattern = r'\bdisproved\b'
    unknown_pattern = r'\bunknown\b'
    has_proved = re.search(proved_pattern, text) is None
    has_disproved = re.search(disproved_pattern, text) is None
    has_unknown = re.search(unknown_pattern, text) is None
    return has_proved and has_disproved and has_unknown
modified_rows = []
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    countb = 0
    countp = 0
    countd = 0
    countu = 0
    counta = 0
    countn = 0
    for index, row in enumerate(reader, start=1):
        column_value = row[column_name]
        if contains_all_three(column_value):
            counta +=1
        if contains_none(column_value):
            countn +=1
        if contains_both_proved_disproved(column_value):
            countb +=1
        if contains_proved(column_value):
            countp +=1
            row[column_name] = "proved"
        if contains_disproved(column_value):
            countd +=1
            row[column_name] = "disproved"
        if contains_unknown(column_value):
            countu +=1
            row[column_name] = "unknown"
        modified_rows.append(row)
    print("all three: ",counta)
    print("none: ",countn)
    print("both p and d: ",countb)
    print("proved: ",countp)
    print("disproved: ",countd)
    print("unknown: ",countu)
    print("total: ",len(modified_rows))
with open("/content/drive/MyDrive/gemini/data/Gemini_parsed_decomp_Gemini_sol.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
    writer.writeheader()
    writer.writerows(modified_rows)

all three:  0
none:  0
both p and d:  0
proved:  240
disproved:  441
unknown:  319
total:  1000


##Checked the one didn't contain any of the three for subs

In [73]:
file_path = '/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv'
column_name = 'answer'

with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for index, row in enumerate(reader):
        if index ==432:
            print(row[column_name])


disproved


## change value for index

In [72]:
df = pd.read_csv('/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv')
df.at[432, 'answer'] = "disproved"
df.to_csv('/content/drive/MyDrive/gemini/data/Gemini_decomp_Gemini_sol.csv', index=False)

## Recheck the answer for subs

In [ ]:
import csv

file_path = '/content/drive/MyDrive/gemini/data/updated_final_results.csv'
column_name = 'answer'

with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    count = 1
    for row in reader:
        print(str(count) + ": " + row[column_name])
        count +=1

# Accuracy check

## Using subs

In [76]:
dataPath = "/content/drive/MyDrive/gemini/data/output-with-subproblems.json"
dataDf = pd.read_json(dataPath, dtype=str)
file_path = '/content/drive/MyDrive/gemini/data/updated_final_results.csv'
column_name = 'answer'
label = []
predicted = []
for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
    label.append(row['label'])
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        predicted.append(row[column_name])
print(len(predicted))

100%|██████████| 1000/1000 [00:00<00:00, 10773.41it/s]


1000


In [77]:
if len(label) == len(predicted):
    matches = sum(1 for label_value, predicted_value in zip(label, predicted) if label_value == predicted_value)
    accuracy = matches / len(label)
    print(f"Accuracy using subproblems: {accuracy:.2%}")
else:
    print("Error: The lists 'label' and 'predicted' have different lengths.")

Accuracy using subproblems: 51.80%


## The base answer

In [ ]:
file_path2 = '/content/drive/MyDrive/gemini/data/updated_gemini_results.csv'
column_name = 'Response'
predicted2 = []
with open(file_path2, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        predicted2.append(row[column_name])
print(len(predicted2))

1000


In [ ]:
if len(label) == len(predicted2):
    matches = sum(1 for label_value, predicted_value in zip(label, predicted2) if label_value == predicted_value)
    accuracy = matches / len(label)
    print(f"Accuracy without using subproblems: {accuracy:.2%}")
else:
    print("Error: The lists 'label' and 'predicted' have different lengths.")

Accuracy without using subproblems: 45.20%


## The CoT answer

In [82]:
file_path3 = '/content/drive/MyDrive/gemini/data/Gemini_parsed_CoT_results.csv'
column_name = 'Response'
predicted3 = []
with open(file_path3, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        predicted3.append(row[column_name])
print(len(predicted3))

1000


In [83]:
if len(label) == len(predicted3):
    matches = sum(1 for label_value, predicted_value in zip(label, predicted3) if label_value == predicted_value)
    accuracy = matches / len(label)
    print(f"Accuracy using CoT: {accuracy:.2%}")
else:
    print("Error: The lists 'label' and 'predicted' have different lengths.")

Accuracy using CoT: 42.90%


## gemini-decomp-gemini-solver

In [78]:
file_path3 = '/content/drive/MyDrive/gemini/data/Gemini_parsed_decomp_Gemini_sol.csv'
column_name = 'answer'
predicted3 = []
with open(file_path3, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        predicted3.append(row[column_name])
print(len(predicted3))

1000


In [80]:
if len(label) == len(predicted3):
    matches = sum(1 for label_value, predicted_value in zip(label, predicted3) if label_value == predicted_value)
    accuracy = matches / len(label)
    print(f"Accuracy using gemini-decomp-gemini-solver: {accuracy:.2%}")
else:
    print("Error: The lists 'label' and 'predicted' have different lengths.")

Accuracy using gemini-decomp-gemini-solver: 50.30%


# parse answer

In [ ]:
import pandas as pd
import re

df = pd.read_csv('/content/drive/MyDrive/gemini/data/Gemini_sub_generation.csv')  # Replace with the actual path to your CSV file

#separates the questions with '||'
def separate_questions(text):
    questions_with_numbers = re.findall(r'(\d+: .*?\?)', text)
    return ' || '.join(questions_with_numbers)

df['parsed_Response'] = df['Response'].apply(separate_questions)
df.to_csv('/content/drive/MyDrive/gemini/data/Gemini_parsed_sub_generation.csv', index=False)  # Replace with your desired output file path

In [ ]:
import pandas as pd
import json

# Load and prepare the JSON data
with open('/content/drive/MyDrive/gemini/data/output-with-subproblems.json', 'r') as file:
    json_data = json.load(file)
columns_needed = ['context', 'question']
new_columns_df = pd.DataFrame(json_data)[columns_needed]

# Load the original CSV
original_df = pd.read_csv('/content/drive/MyDrive/gemini/data/Gemini_parsed_sub_generation.csv')

# Concatenate and save
updated_df = pd.concat([original_df, new_columns_df], axis=1)
updated_df.to_csv('/content/drive/MyDrive/gemini/data/Gemini_parsed_sub_generation.csv', index=False)

In [ ]:
import pandas as pd
pa = '/content/drive/MyDrive/gemini/data/Gemini_parsed_sub_generation.csv'
# Load the CSV into a DataFrame
df = pd.read_csv(pa)

# Delete the "Example" column
df = df.drop('Example', axis=1)

# Save the modified DataFrame back to the CSV
df.to_csv(pa, index=False)